In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/comment/1.csv


    # (मैंने नहीं रूपया भेजना है, नहीं क्यो आर देने है): 0   these are negative statements (expresses a negation, denial, disagreement, or falsehood)
    #  मैंने रूपया भेजना है :  1                     statements meant to send money/make payment
    # (मैं रूपया लेने है, क्यो आर देना है) : 2          statements meant to accept payment or ask qr code, phone number or other credentials
    # अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें : 3  these are random statements

In [4]:
df = pd.read_csv('/kaggle/input/comment/1.csv')

,comment,value
0,मैंने रूपया भेजना है,1
1,मैं रूपया भेजना,1
2,मैं रूपया भेजना कर दीजिये,1
3,मैं रूपया भेजना दीजिये,1
4,मैं रूपया भेजना करना है,1
...,...,...
494376,आज युवा आशा और आकांक्षाओं से भरे हुए हैं।,3
494377,करते रहे विलाप होकर निःशब्द,3
494378,आपको सफलता मिलेगी परंतु कभी-कभी असफलता भी मिल ...,3
494379,आप अपनी जीवनवृत्ति में अनेक पेशेवर उपलब्धियां ...,3


In [5]:
print("total duplicate statements:",df.duplicated().sum())
print("value distribution across various labels: ",df['value'].value_counts())
df.drop_duplicates(keep='first',inplace=True)
df = df.reset_index(drop=True)

total duplicate statements: 132271
value distribution across various labels:  value
3    359993
0    105251
2     20136
1      9001
Name: count, dtype: int64


In [15]:
print("value distribution across various labels after droping duplicates: ",df['value'].value_counts())

value distribution across various labels after droping duplicates:  value
3    256025
0     83078
2     16206
1      6801
Name: count, dtype: int64


    # class 3 which is random statement is a lot more than others, we are just taking a part of it  , 50000 instances

In [35]:
new_df = df.iloc[0:150000,:]
new_df = new_df.reset_index(drop=True)
new_df['value'].value_counts()

value
0    83078
3    43915
2    16206
1     6801
Name: count, dtype: int64

**PreProcessing**

In [16]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [7]:
from tqdm import tqdm
import re
import string,time
exclude = string.punctuation

def preprocess(text):
    word_list = text.strip().split()
    text = " ".join(word_list)
    clean = re.compile('<.*?>')  # Pattern to match HTML tags
    text =  re.sub(clean, '', text)  # Remove HTML tags from the text
  # Regular expression pattern to match URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    # Replace URLs with an empty string
    text = url_pattern.sub('', text)
    return text.translate(str.maketrans('','',exclude))


In [8]:
tqdm.pandas(desc="Processing Comments")
new_df['comment'] = new_df['comment'].progress_apply(preprocess)

Processing Comments: 100%|██████████| 160000/160000 [00:02<00:00, 65726.25it/s]


In [9]:
new_df["value"].value_counts()

value
0    83078
3    53915
2    16206
1     6801
Name: count, dtype: int64

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize

The CountVectorizer was not correctly handling non-ASCII characters in the input text, resulting in unexpected behavior when trying to represent the bag-of-words for the text.
Possible Causes:

The default tokenization and encoding methods used by CountVectorizer were not sufficient for processing non-ASCII characters.
Solution:

The solution involved providing a custom tokenizer (word_tokenize from the nltk library) through the tokenizer parameter of CountVectorizer. This custom tokenizer handles non-ASCII characters correctly during tokenization.
Additionally, the encoding='utf-8' parameter was added to ensure proper encoding of the text.

In [11]:
# Create CountVectorizer object with a custom tokenizer
questions = list(new_df['comment'])
cv = CountVectorizer(tokenizer=word_tokenize, encoding='utf-8',ngram_range=(2, 3),max_features=4000)
cv = cv.fit(questions)
feature_names = cv.get_feature_names_out()
len(feature_names)

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


4000

In [12]:
q1 = cv.transform(new_df['comment']).toarray()
print(q1.shape)
q1

(160000, 4000)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [42]:
temp_df = pd.DataFrame(q1,index = new_df.index)
temp_df

,0,1,2,3,4,5,6,7,8,9,...,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
159996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
159997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
159998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


    # Creating a new feature , wheter the sentence contain -ve word or not

In [43]:
word_neg = ['नहीं']
def fun1(chat):
    word_list = chat.strip().split()
    for word in word_list:
        if word  in word_neg:
            return 1
    return 0

In [44]:
tqdm.pandas(desc="Processing Comments")
temp_df['word_neg'] = new_df['comment'].progress_apply(fun1)
temp_df['value'] = new_df['value']
temp_df.head()

Processing Comments: 100%|██████████| 160000/160000 [00:00<00:00, 279455.62it/s]


,0,1,2,3,4,5,6,7,8,9,...,3992,3993,3994,3995,3996,3997,3998,3999,word_neg,value
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


**Model Building**

In [46]:
temp_df.columns = temp_df.columns.astype(str)

In [47]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1],temp_df.loc[:,'value'],test_size=0.2,random_state=1)

In [50]:
print(X_train.shape )
print(X_test.shape )
X_train.head()

(128000, 4001)
(32000, 4001)


,0,1,2,3,4,5,6,7,8,9,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,word_neg
13841,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
79526,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
106100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132321,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Building A Random Forest Classifier**

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score , confusion_matrix
rf = RandomForestClassifier(verbose=2)

rf_classifier = RandomForestClassifier(n_estimators=100)

Set verbose to control the output
rf_classifier.verbose = 0 

In [52]:
rf.fit(X_train,y_train)

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  2.8min


building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82

RandomForestClassifier(verbose=2)

In [23]:
rf.verbose = 0 
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.99778125

In [29]:
# Calculate precision, recall, and F1-score for each class
precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred, average=None)

# Display results for each class
for i, (p, r, f, s) in enumerate(zip(precision, recall, f1, support)):
    print(f"Class {i}:")
    print(f"  Precision: {p:.2f}")
    print(f"  Recall: {r:.2f}")
    print(f"  F1-Score: {f:.2f}")
    print(f"  Support: {s}")

# Overall accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nOverall Accuracy: {accuracy:.2f}")

Class 0:
  Precision: 1.00
  Recall: 1.00
  F1-Score: 1.00
  Support: 16580
Class 1:
  Precision: 0.98
  Recall: 0.97
  F1-Score: 0.98
  Support: 1343
Class 2:
  Precision: 0.99
  Recall: 0.99
  F1-Score: 0.99
  Support: 3304
Class 3:
  Precision: 1.00
  Recall: 1.00
  F1-Score: 1.00
  Support: 10773

Overall Accuracy: 1.00
